<a href="https://colab.research.google.com/github/guilherme-daros/EEL7123/blob/master/trab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo de carregamento de imagens


In [ ]:
import os

import tensorflow as tf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cria DataFrame com todos os caminhos das imagens e respectivos rótulos
base_path = 'drive/MyDrive/Dataset'
df_all = pd.DataFrame()
for dirname, _, filenames in os.walk(base_path):
    paths = [dirname + '/' + filename for filename in filenames if '.jpg' in filename]
    folder_name = os.path.split(dirname)[-1]
    df_all = pd.concat([df_all, pd.DataFrame({'path': paths, 'label': folder_name})], ignore_index=True)

df_all

,path,label
0,drive/MyDrive/Dataset/Garbage classification/c...,cardboard
1,drive/MyDrive/Dataset/Garbage classification/c...,cardboard
2,drive/MyDrive/Dataset/Garbage classification/c...,cardboard
3,drive/MyDrive/Dataset/Garbage classification/c...,cardboard
4,drive/MyDrive/Dataset/Garbage classification/c...,cardboard
...,...,...
2522,drive/MyDrive/Dataset/Garbage classification/t...,trash
2523,drive/MyDrive/Dataset/Garbage classification/t...,trash
2524,drive/MyDrive/Dataset/Garbage classification/t...,trash
2525,drive/MyDrive/Dataset/Garbage classification/t...,trash


In [ ]:
df_all['label'].value_counts().sort_index()

cardboard    403
glass        501
metal        410
paper        594
plastic      482
trash        137
Name: label, dtype: int64

In [ ]:
classes = list(df_all['label'].unique())
n_classes = len(classes)

In [ ]:
# Reduz o conjunto para 125 imagens por classe

df = df_all.iloc[:0]
for t in classes:
    subset = df_all[df_all['label'] == t]
    subset = subset.sample(125, random_state=0)
    df = pd.concat([df, subset], ignore_index=True)

df['label'].value_counts().sort_index()

cardboard    125
glass        125
metal        125
paper        125
plastic      125
trash        125
Name: label, dtype: int64

In [ ]:
# Separa entre treinamento (completo e reduzido), validação e teste, sempre com número uniforme de imagens por classe
df_train, df_test = train_test_split(df, test_size=25*n_classes, stratify=df['label'], random_state=42)
df_train, df_val  = train_test_split(df_train, test_size=25*n_classes, stratify=df_train['label'], random_state=42)
df_train_small, _ = train_test_split(df_train, train_size=25*n_classes, stratify=df_train['label'], random_state=42)

In [ ]:
# Função para preparar o dataset
try:
    AUTOTUNE = tf.data.AUTOTUNE     
except:
    AUTOTUNE = tf.data.experimental.AUTOTUNE 

def create_dataset(df, img_size, batch_size):
  paths = df['path']
  labels = df['label'].map(classes.index) # ordinal encoder
  ds = tf.data.Dataset.from_tensor_slices((paths, labels))
  ds = ds.map(lambda x, y: (tf.io.read_file(x), y))
  ds = ds.map(lambda x, y: (tf.io.decode_jpeg(x, channels=3), y))
  ds = ds.map(lambda x, y: (tf.image.resize(x, img_size), y))
  ds = ds.cache().batch(batch_size).prefetch(AUTOTUNE)
  return ds

In [ ]:
# Hiperparâmetros configuráveis

img_size = (299, 299)  
batch_size = 32

ds_train_small =  create_dataset(df_train_small, img_size, batch_size)
ds_train =        create_dataset(df_train, img_size, batch_size)
ds_val =          create_dataset(df_val, img_size, batch_size)
ds_test =         create_dataset(df_test, img_size, batch_size)

print('------------------------------')
print('datasets division')
print('------------------------------')
print(f'# train (small) = {len(df_train_small)}')
print(f'# train (full)  = {len(df_train)}')
print(f'# val           = {len(df_val)}')
print(f'# test          = {len(df_test)}')

------------------------------
datasets division
------------------------------
# train (small) = 150
# train (full)  = 450
# val           = 150
# test          = 150


### Questão 2
A.i)


In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.xception import Xception, preprocess_input


# Hiperparâmetros
input_shape = (299,299,3)
epochs = 10
lr = 0.01

model1 = Xception(include_top=False, input_shape=input_shape, pooling='avg')
model1.trainable = True

i = Input(shape=input_shape)
x = i
x = preprocess_input(x)
x = model1(x, training=False)

x = Dropout(0.2)(x)
x = Dense(6, activation='softmax')(x)

model1 = Model(i, x)
model1.summary()

model1.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"], optimizer=Adam(learning_rate=lr))
history = model1.fit(ds_train_small, validation_data=ds_val, epochs=epochs)

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 299, 299, 3)]     0         
                                                                 
 tf.math.truediv_13 (TFOpLam  (None, 299, 299, 3)      0         
 bda)                                                            
                                                                 
 tf.math.subtract_13 (TFOpLa  (None, 299, 299, 3)      0         
 mbda)                                                           
                                                                 
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 dropout_13 (Dropout)        (None, 2048)              0         
                                                                 
 dense_13 (Dense)            (None, 6)                 122

A.ii)


In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.xception import Xception, preprocess_input


# Hiperparâmetros
input_shape = (299,299,3)
epochs = 10
lr = 0.001

model1 = Xception(include_top=False, input_shape=input_shape, pooling='avg')
model1.trainable = False

i = Input(shape=input_shape)
x = i
x = preprocess_input(x)
x = model1(x, training=False)

x = Dropout(0.2)(x)
x = Dense(6, activation='softmax')(x)

model1 = Model(i, x)
model1.summary()

model1.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"], optimizer=Adam(learning_rate=lr))
history = model1.fit(ds_train_small, validation_data=ds_val, epochs=epochs)

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 299, 299, 3)]     0         
                                                                 
 tf.math.truediv_9 (TFOpLamb  (None, 299, 299, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_9 (TFOpLam  (None, 299, 299, 3)      0         
 bda)                                                            
                                                                 
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 dropout_9 (Dropout)         (None, 2048)              0         
                                                                 
 dense_9 (Dense)             (None, 6)                 1229

A.iii)


In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.xception import Xception, preprocess_input


# Hiperparâmetros
input_shape = (299,299,3)
epochs = 10
lr = 1e-5

model1.trainable = True

model1.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"], optimizer=Adam(learning_rate=lr))
history = model1.fit(ds_train_small, validation_data=ds_val, epochs=epochs)

Epoch 1/10
5/5 [==============================] - 12s 2s/step - loss: 0.0654 - accuracy: 1.0000 - val_loss: 0.7952 - val_accuracy: 0.7067
Epoch 2/10
5/5 [==============================] - 7s 1s/step - loss: 0.0359 - accuracy: 1.0000 - val_loss: 0.7852 - val_accuracy: 0.7067
Epoch 3/10
5/5 [==============================] - 7s 1s/step - loss: 0.0258 - accuracy: 1.0000 - val_loss: 0.7917 - val_accuracy: 0.7000
Epoch 4/10
5/5 [==============================] - 7s 1s/step - loss: 0.0129 - accuracy: 1.0000 - val_loss: 0.8149 - val_accuracy: 0.7000
Epoch 5/10
5/5 [==============================] - 7s 1s/step - loss: 0.0087 - accuracy: 1.0000 - val_loss: 0.8352 - val_accuracy: 0.7000
Epoch 6/10
5/5 [==============================] - 7s 1s/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.8457 - val_accuracy: 0.7000
Epoch 7/10
5/5 [==============================] - 7s 1s/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.8531 - val_accuracy: 0.7067
Epoch 8/10
5/5 [========================